# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_file = "../WeatherPy/output_data/cities.csv"

city_weather = pd.read_csv(input_file)
del city_weather['Unnamed: 0']
city_weather


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,100,NZ,1586643300,65,-46.19,168.86,57.99,4.00
1,tautira,100,PF,1586643300,66,-17.73,-149.15,87.80,3.36
2,cockburn town,0,TC,1586643300,69,21.46,-71.14,80.69,10.11
3,miri,75,MY,1586643300,100,4.41,114.01,77.00,2.24
4,caravelas,62,BR,1586643300,85,-17.71,-39.25,79.21,11.05
...,...,...,...,...,...,...,...,...,...
568,oistins,40,BB,1586644010,69,13.07,-59.53,80.60,12.75
569,kashi,98,CN,1586643941,18,39.45,75.98,59.00,4.47
570,ostrovnoy,51,RU,1586643625,93,68.05,39.51,15.75,11.48
571,longyearbyen,20,SJ,1586643981,76,78.22,15.64,-7.60,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_weather[["Lat", "Lng"]]

humidity = city_weather["Humidity"]

figure_layout = {
    'width': '1200px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Plot Heatmap


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_locations = city_weather.loc[(city_weather['Max Temp'] >= 70) & (city_weather['Max Temp'] < 80), :]
ideal_locations = ideal_locations.loc[city_weather['Wind Speed'] < 10,:]
ideal_locations = ideal_locations.loc[city_weather['Cloudiness'] == 0,:]
ideal_locations.dropna(inplace=True)
ideal_locations

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
139,pisco,0,PE,1586643437,30,-13.70,-76.22,75.99,5.82
187,ambilobe,0,MG,1586643551,84,-13.20,49.05,73.67,4.72
210,bella union,0,UY,1586643561,49,-30.28,-57.60,72.23,5.41
281,namibe,0,AO,1586643420,61,-15.20,12.15,77.59,3.62
285,formosa,0,AR,1586643572,56,-26.18,-58.18,71.60,3.36
315,benguela,0,AO,1586643528,82,-12.58,13.41,78.75,4.88
334,muddebihal,0,IN,1586643747,24,16.33,76.13,78.91,8.84
345,chakulia,0,IN,1586643747,40,22.48,86.72,78.57,3.87
353,narasannapeta,0,IN,1586643748,88,18.42,84.05,78.06,5.46
554,tura,0,IN,1586643479,32,25.52,90.22,72.18,4.45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_locations
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lng'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
139,pisco,0,PE,1586643437,30,-13.70,-76.22,75.99,5.82,,,
187,ambilobe,0,MG,1586643551,84,-13.20,49.05,73.67,4.72,,,
210,bella union,0,UY,1586643561,49,-30.28,-57.60,72.23,5.41,,,
281,namibe,0,AO,1586643420,61,-15.20,12.15,77.59,3.62,,,
285,formosa,0,AR,1586643572,56,-26.18,-58.18,71.60,3.36,,,
315,benguela,0,AO,1586643528,82,-12.58,13.41,78.75,4.88,,,
334,muddebihal,0,IN,1586643747,24,16.33,76.13,78.91,8.84,,,
345,chakulia,0,IN,1586643747,40,22.48,86.72,78.57,3.87,,,
353,narasannapeta,0,IN,1586643748,88,18.42,84.05,78.06,5.46,,,
554,tura,0,IN,1586643479,32,25.52,90.22,72.18,4.45,,,


In [6]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key
}


# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [7]:
hotel_results = hotel_df.loc[hotel_df["Hotel Name"] != "",:]
hotel_results

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
139,pisco,0,PE,1586643437,30,-13.70,-76.22,75.99,5.82,PiscoMar Peru,-13.7136,-76.2188
187,ambilobe,0,MG,1586643551,84,-13.20,49.05,73.67,4.72,Hotel National,-13.1959,49.0493
210,bella union,0,UY,1586643561,49,-30.28,-57.60,72.23,5.41,Hotel Tres Fronteras~Monte Caseros,-30.2533,-57.6226
281,namibe,0,AO,1586643420,61,-15.20,12.15,77.59,3.62,Hotel ibis Styles Iu Namibe,-15.1909,12.1512
285,formosa,0,AR,1586643572,56,-26.18,-58.18,71.60,3.36,Howard Johnson Hotel And Casino Formosa,-26.1962,-58.2036
315,benguela,0,AO,1586643528,82,-12.58,13.41,78.75,4.88,iu Hotel Benguela,-12.6034,13.4032
334,muddebihal,0,IN,1586643747,24,16.33,76.13,78.91,8.84,Hotel Annapurna,16.3357,76.1259
345,chakulia,0,IN,1586643747,40,22.48,86.72,78.57,3.87,Debi Rani Debi Lodge,22.4794,86.7258
353,narasannapeta,0,IN,1586643748,88,18.42,84.05,78.06,5.46,Sri laxmi lodge,18.4157,84.0411
554,tura,0,IN,1586643479,32,25.52,90.22,72.18,4.45,Hotel Polo Orchid,25.5145,90.1922


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_results.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
locations
# Add marker layer ontop of heat map

coordinates = list(zip(hotel_results["Lat"],hotel_results["Lng"]))

markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…